<a href="https://colab.research.google.com/github/Ebratul/Python/blob/main/NLP/NLP1_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: g65ashaharier21031
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 2.61GB/s]

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df['review'][5]

'Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.'

In [7]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [8]:
df.isnull().sum()

,0
review,0
sentiment,0


In [9]:
df.duplicated().sum()

np.int64(418)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
#Basic preprocessing
#remove tags
#Lowercase
#remove stopwords

In [13]:
import re
def remove_tags(raw_text):
  clean_text = re.sub(re.compile('<.*?>'),'', raw_text)
  return clean_text

In [14]:
df['review'] = df['review'].apply(remove_tags)

In [15]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [16]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [17]:
import nltk
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
49995,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [20]:
df['review'][5]

'probably all-time favorite movie, story selflessness, sacrifice dedication noble cause, preachy boring. never gets old, despite seen 15 times last 25 years. paul lukas\' performance brings tears eyes, bette davis, one truly sympathetic roles, delight. kids are, grandma says, like "dressed-up midgets" children, makes fun watch. mother\'s slow awakening what\'s happening world roof believable startling. dozen thumbs, "up" movie.'

In [21]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [22]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
49995,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [23]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [25]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [28]:
x_train.shape

(39665, 1)

In [29]:
#apply BOW
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv = CountVectorizer()

In [31]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [34]:
x_train_bow.shape

(39665, 94644)

In [35]:
x_test_bow.shape

(9917, 94644)

In [36]:
from sklearn.naive_bayes import GaussianNB

In [37]:
gnb = GaussianNB()

In [38]:
gnb.fit(x_train_bow, y_train)

GaussianNB()

In [39]:
y_pred = gnb.predict(x_test_bow)
y_pred

array([0, 1, 1, ..., 0, 0, 1])

In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_pred)

0.6408187960068569

In [41]:
confusion_matrix(y_test, y_pred)

array([[4009,  930],
       [2632, 2346]])

In [42]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_bow, y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test, y_pred)

0.8529797317737219